In [11]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [16]:
def ingest_data(path):
    data = pd.read_csv(path)
    data.dropna(inplace=True)
    data.reset_index(inplace=True)
    data.drop('index', axis=1, inplace=True)
    return data

In [17]:
def clean_data(data):
    stopwords = nltk.corpus.stopwords.words('english')
    lem=WordNetLemmatizer()
    pass

In [25]:
if __name__ == "__main__":
    ingested_data = ingest_data('Data/fake-news/train.csv')
    X=ingested_data.drop(['id', 'label'], axis=1)
    Y=ingested_data['label']
    ID=ingested_data['id']
    cleaned_data = clean_data(X)

0            0
1            1
2            2
3            3
4            4
         ...  
18280    20795
18281    20796
18282    20797
18283    20798
18284    20799
Name: id, Length: 18285, dtype: int64